# Handling Errors in Python

Unfortunately, things go wrong. When software goes wrong, it's important that your program gives your users, their support staff, or you and your software engineering colleagues enough information to know _what_ went wrong, _how_ it went wrong, and _what_ (if anything) can be done about it.

## The Errorgraphie

The event of software going wrong is called an error. We can define a taxonomy of the kinds of errors your program will encounter: what you do in response to each kind will be different.

### Disappointments

Something that could reasonably be expected to go wrong did in fact go wrong. A user asked your program to open a file that doesn't exist, a network resource was unavailable, or the credentials to connect to a database being incorrect are all examples of this kind of error, which we could call a "disappointment". Usually the best thing to do is to give the user a clear description of what went wrong, along with remedial action to take before retrying the operation.

### Precondition Failures

Somebody tried to use your code in a way that you don't support. For example, perhaps your maths library only supports real numbers, but a programmer passed a negative number to the square root function; a programmer tried to pop a value from an empty stack; or the `start()` method was called on your simulation object before the initial parameters were configured.

Often there is no way to recover from this problem, and your program must terminate. Give specific information about what you expected and why this expectation was not met, so that the programmer can understand how they need to fix their code to integrate with yours correctly.

### Postcondition Failures

If your code is called correctly, nothing surprising or disappointing happens, and your program _still_ doesn't generate a satisfactory result, then you have a bug. It's on you to fix it. We'll see techniques in this section to avoid introducing bugs and to diagnose their fixes, but as with the earlier types of error if you can detect that your code is going wrong, it's a good idea to give specific information about why to aid in diagnosis.

### Underlying System Errors

Your code works perfectly, it was called correctly, everything went as expected...then the operating system killed your process. Such problems are unavoidable (to a first approximation, anyway: if your process was killed because the computer ran out of memory, you could consider whether you're trying to use too much) and unrecoverable. Unsurprisingly, I still recommend giving as much specific information about the failure as possible so that people can see _that_ your program failed and _why_, and consider what if anything is to be done about it.

## Error handling in Python

While the four classes of errors in The Errorgraphie communicate different problems, Python only has one method for signalling and handling errors: the exception. You can see the effect of raising an exception in a notebook by doing something that will go wrong.

In [ ]:
1/0

The exception is of kind `ZeroDivisionError` and has an explanation message ("division by zero"); together these help you to understand the error that occurred.

### Handling errors in code

Your program can "catch" an exception to be notified that it happened and do something in response. Enclose the code that might cause an error in a `try:` block. After the `try:` block, you can have one or more `except:` blocks. If you name a type of error in an `except:` block, that block is only executed if an exception of that type or a subtype is raised. Here's our attempt to divide by zero again, with error handling.

In [ ]:
try:
    1/0
except ZeroDivisionError:
    print("Couldn't calculate the answer, it involved dividing by zero.")
except:
    print("An unknown error occurred.")


Notice that the error didn't result in a traceback in Jupyter. Handling the error in `except:` "swallows" it. If you want to indicate to calling code that an error occurred, even though you caught it in `except:` yourself, you can `raise` it again.

In [ ]:
try:
    1/0
except:
    print("Something went wrong!")
    raise


### Raising your own errors

Your code should detect erroneous situations and give specific errors, described in the terms relevant to your problem domain. This makes it easier for users to understand what went wrong and take appropriate steps to resolve the issue. Even if those steps turn out to be raising a bug report on your project, you'll benefit from more detailed information about what went wrong.

Let's take the example of calculating the mean of some values in a collection. This could go wrong if the collection is empty, because then the sum (0, if the collection represents numbers or number-like) will be divided by 0, the count of elements in the collection. Rather than raise the `ZeroDivisionError` we've already seen, it'd be helpful to raise an error specifically calling out the empty collection as the cause.

To do so, we must do two things. The first is to define the error type, which is a class that inherits from the built-in `Exception` type (or a more specific subclass).

In [ ]:
class EmptyCollectionError(Exception):
    pass


Then in your own code, you should `raise` an instance of this error.

In [ ]:
def mean(collection):
    if len(collection) == 0:
        raise EmptyCollectionError()
    return sum(collection)/len(collection)

In [ ]:
# This should work without problem
print(mean([1,2,3,4]))

# This should raise our custom exception type
try:
    mean_of_nothing = mean([])
    print(f"Mean of an empty list is {mean_of_nothing}")
except EmptyCollectionError:
    print("Tried to get mean of an empty collection")
except ZeroDivisionError:
    # We won't get here, we avoided triggering the built-in error
    print("Divided by zero")
except:
    print("Another, different error occurred")

You could add context to your custom error type, for example by adding parameters to its constructor that identify the error in more detail. To inspect the error, an `except:` block can assign the caught exception to a variable. In the following code cell, we add more context to the `EmptyCollectionError` showing what type of empty collection was passed.

In [ ]:
class EmptyCollectionError(Exception):
    def __init__(self, collection):
        self.type = type(collection)

def mean(collection):
    if len(collection) == 0:
        raise EmptyCollectionError(collection)
    return sum(collection)/len(collection)

try:
    mean_of_nothing = mean([])
    print(f"Mean of an empty list is {mean_of_nothing}")
except EmptyCollectionError as error:
    print(f"Collection of type {error.type} was empty")

### Success and `else:` blocks

After the `try:` block has been executed, if no exception occurs, then Python will pass control to an `else:` block if it exists. In other words, this block gets executed if (and only if) the code in the `try:` does not raise an error. This may appear to be redundant, as the code could go at the end of the `try:` block and would run only if the preceding code did not raise. While that argument is correct, the `else:` block exists to aid in designing clear error-handling code.

By moving code from the end of the `try:` to an `else:` block, you communicate to readers which part of the code the error-handling specifically relates to. You also ensure that any changes in the follow-on code that introduce errors are not accidentally absorbed by your existing `except:` clauses.

In [ ]:
try:
    mean_value = mean([2, 3, 5, 7, 11])
except EmptyCollectionError:
    print("Collection was empty")
else:
    print(f"mean of the first five primes is {mean_value}")

### Cleaning up afterwards

Often you will have clean-up action to complete whether or not an operation failed. For example, if you opened a file to read input data and an error happened while you were working on that input, you'd want to close the file. But you'd _also_ want to close the file if everything succeeded.

Such clean-up code goes in a `finally:` block, which follows all of the `except:` error handlers and the `else:` block. It will run after the code in the `try:` block, whether or not an exception is raised (and whether or not that exception is handled locally in an `except:`).

In [ ]:
try:
    f = open('input.txt')
    message = f.readline()
except:
    print("Something didn't go right...")
    raise
else:
    print(message)
finally:
    print("Closing file")
    f.close()

There are so many examples of objects that acquire some resource on creation that needs cleaning up regardless of whether an error occurs that Python has specific syntax to handle the case. This is out of scope for the course, but you can refer to [the `with` statement](https://docs.python.org/3/reference/compound_stmts.html#with) and [Context Manager types](https://docs.python.org/3/library/stdtypes.html#typecontextmanager) in the Python documentation.